# Basic exploration of the dataset
Here we look at the basic exploration of the fraud detection dataset.

In [ ]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
spark = SparkSession.builder.config("spark.driver.memory","30g").getOrCreate()

In [ ]:
REL_DATA_PATH = "../data"
TRAIN_TRANSACTION_PATH=f"{REL_DATA_PATH}/train_transaction.csv"
TRAIN_IDENTITY_PATH=f"{REL_DATA_PATH}/train_identity.csv"
TEST_TRANSACTION_PATH=f"{REL_DATA_PATH}/test_transaction.csv"
TEST_IDENTITY_PATH=f"{REL_DATA_PATH}/test_identity.csv"

# Looking at the training data

In [ ]:
train_data = spark.read.csv(TRAIN_TRANSACTION_PATH,header=True, inferSchema=True)

In [ ]:
train_data.dtypes

In [ ]:
train_data.limit(20).toPandas()

In [ ]:
train_data.describe().toPandas()

# Looking at individual features
## is Fraud
This seems to be the ground truth or `target` label. 

In [ ]:
px.bar(train_data.groupBy("isFraud").count().toPandas(),x='isFraud',y='count', title="Count of transactions by type")

## Transaction DateTime
Looking at the `TransactionDT` column. Largely seems to be masked by  some integer conversion. 

In [ ]:
train_data.select("TransactionDT").distinct().count()


In [ ]:
train_data.groupby('TransactionDT').count().sort('count', ascending=False).toPandas()

Seems like most of the timestamps are unique. So Not many more than one transaction happening at once.

>Might be worthwhile looking to see if the count of fraud is different for transaction events that overlap with others.

## TransactionAmt
This seems to be the transaction amount.

In [ ]:
px.histogram(train_data.select(F.col("TransactionAmt").astype('float')).sort("TransactionAmt").toPandas(),x='TransactionAmt', nbins=20)

Seems to have a very long tail. Also seems to be negative values?

In [ ]:
train_data.select("TransactionAmt").where(F.col("TransactionAmt").astype('float') < 0).toPandas()

Nope seems to be all good. Looks like most of the values are distributed aroudn 0 and 5000, let's look at a closer histogram there.

In [ ]:
px.histogram(train_data.where(F.col("TransactionAmt") < 5000).select(F.col("TransactionAmt").astype("float")).toPandas(), x="TransactionAmt", nbins=100)

Looks liek most of the transactions are peaked in the 25-75. 

## ProductCD
Looks like product code, assuming that this is the product code of the thing that was pruchased?

In [ ]:
px.histogram(train_data.select("ProductCD").toPandas(), x='ProductCD')

In [ ]:
px.histogram(train_data.select("ProductCD","isFraud").groupBy("ProductCD","isFraud").count().toPandas(), x="ProductCD", color="isFraud", y="count")

Looks like `C` has a very high % of fraud compared to the others.

## card1
Seems to be a integer field.


In [ ]:
px.histogram(train_data.select("card1").toPandas(), x="card1", nbins=100)

In [ ]:
def get_null_count(d, c):
    """
    Returns a pandas dataframe with the count of nulls
    """
    return d.select(c).withColumn("isNull",F.col(c).isNull()).groupby("isNull").count().toPandas()

In [ ]:
get_null_count(train_data,"card1")

Seems to not have any nulls. 

# card2
is a floating point value.


In [ ]:
px.histogram(train_data.select("card2").toPandas(), x="card2", nbins=100)

In [ ]:
get_null_count(train_data,"card2")

Seems like there's a few nulls.

## card3
This is also a double field

In [ ]:
px.histogram(train_data.select("card3").toPandas(), x="card3", nbins=100)

Hmm interestingly, it seems to be peaking around 150-151. Probably could leave this feature out of the base models.

In [ ]:
get_null_count(train_data,"card3")

Seems to be there is a small amount of nulls here too.

## card4
This is a string / level column

In [ ]:
px.bar(train_data.select("card4",F.col("isFraud").astype("string")).groupby("card4","isFraud").count().toPandas(), x = "card4", y = "count", color = "isFraud")

Interestingly there are no fraud records for both `discover` and `AE` cards. 

In [ ]:
get_null_count(train_data,"card4")

Small amount of nulls in this case too.

## card5
This is a floating point feature.

In [ ]:
px.histogram(train_data.select("card5").toPandas(), x="card5")

This seems also peaky around 224, 226. Might exlude it from the first attempts at models.

In [ ]:
get_null_count(train_data,"card5")

Bit higher number of nulls than the others. 

## card 6
This is a string column. The feature seems to be about card type.

In [ ]:
px.bar(train_data.select("card6",F.col("isFraud").astype("string")).groupby("card6","isFraud").count().toPandas(), x= "card6", y= "count", color = "isFraud" )

There's more credit card fraud that debit card fraud (proportionally). Which is intuitive I suppose.

In [ ]:
get_null_count(train_data,"card6")

Small amount of nulls here too. 

## addr1
This is a floating point column

In [ ]:
px.histogram(train_data.select("addr1").toPandas(), x= "addr1", nbins = 100)

Looks a bit spikey, but seems to have prettey good range. Might leave out for the first model. 

In [ ]:
get_null_count(train_data,"addr1")

Lots of nulls too. 

## addr2
This is another floating point column

In [ ]:
px.histogram(train_data.select("addr2").toPandas(), x= "addr2", nbins = 100)

Very spikey, I think we'll leave this out too.

In [ ]:
get_null_count(train_data,"addr2")

Similar to addr1 when it comes to nulls it seems.

## P_emaildomain
This is a string column. I believe this is the payee email domain.

In [ ]:
px.bar(train_data.select("P_emaildomain",F.col("isFraud").astype("string")).groupby("P_emaildomain", "isFraud").count().toPandas(), x="P_emaildomain",y="count",color="isFraud")

Gmail, yahoo and hotmail seem to have notably visible fraud. Also anonymous.com as well as aol coming in too. Zooming in, shows outlook has quite high fraud too.

In [ ]:
get_null_count(train_data,"P_emaildomain")

Hmm lot's of nulls here too. Maybe leave it out for first attempt. Can just bucket all the nulls into unknown as a first run though.

## R_emaildomain
String column. I believe this is the recipient email domain.

In [ ]:
px.bar(train_data.select("R_emaildomain",F.col("isFraud").astype("string")).groupby("R_emaildomain","isFraud").count().toPandas(), x = "R_emaildomain", y = "count", color="isFraud")

Similar outcome to P_emaildomain

In [ ]:
get_null_count(train_data,"R_emaildomain")

Seems to be mostly nulls, so probably not use this in early models.

# Summary
There are some simple features to get started with, like transaction amount , product type, etc. Which don't need exotic treatment with nulls. 

We also skipped all the `c` features. We can include them in future investigations.